In [ ]:
# Install packages
!pip install langgraph langchain langchain-core litellm --quiet

print("✅ All packages installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 28.5 MB/s eta 0:00:00
✅ All packages installed!


In [ ]:
# Set up API key
import os
from getpass import getpass

# Get API key from user
if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass('Enter your Groq API key: ')

print("✅ API key configured!")

Enter your Groq API key: ··········
✅ API key configured!


In [ ]:
# Import core libraries
from typing import TypedDict, Annotated, List, Literal
import operator
from litellm import completion
import json

# Helper function to call LLM
def call_llm(prompt: str, system: str = "You are a helpful assistant.") -> str:
    """Simple wrapper for LLM calls."""
    try:
        response = completion(
            model="groq/llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1024
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test it
print(call_llm("Say 'Multi-agent systems are awesome!' in exactly those words."))

Multi-agent systems are awesome!


In [ ]:
# TODO: Define the researcher agent

def researcher_agent(topic: str) -> str:
    """Researches a topic and returns key findings."""
    system = """You are a research specialist. Your job is to:
    1. Identify 3-4 key points about the given topic
    2. Provide factual, concise information
    3. Format as bullet points
    Keep your response under 200 words."""

    # Create the prompt and call the LLM
    prompt = f"Research the topic: {topic}"
    return call_llm(prompt, system)

# Test it
# research = researcher_agent("benefits of meditation")
# print(research)


In [ ]:
# TODO: Define the writer agent

def writer_agent(research: str, topic: str) -> str:
    """Writes content based on research findings."""
    system = """You are a content writer. Your job is to:
    1. Take research findings and create engaging content
    2. Write in a clear, accessible style
    3. Include an introduction and conclusion
    Keep your response under 300 words."""

    # Create prompt that includes the research and topic
    prompt = f"""Topic: {topic}

Research Findings:
{research}

Write a well-structured article based on the above research."""

    return call_llm(prompt, system)

# Test it
# draft = writer_agent(research, "benefits of meditation")
# print(draft)


In [ ]:
# TODO: Define the editor agent

def editor_agent(draft: str) -> str:
    """Edits and polishes content."""
    system = """You are an editor. Your job is to:
    1. Improve clarity and flow
    2. Fix any grammatical issues
    3. Add a compelling title
    Return the polished version."""

    # Create prompt for editing
    prompt = f"""Edit and polish the following draft:

{draft}

Provide a refined version with a clear title."""

    return call_llm(prompt, system)

print("✅ Agents defined!")



✅ Agents defined!


In [ ]:
# TODO: Create the pipeline function

def run_content_pipeline(topic: str) -> dict:
    """Run the full content creation pipeline."""
    print(f"📚 Starting pipeline for: '{topic}'")
    print("=" * 50)

    # Step 1: Research
    print("\n🔍 Step 1: Researcher Agent working...")
    research = researcher_agent(topic)
    print(f"Research complete! ({len(research)} chars)")

    # Step 2: Writing
    print("\n✍️ Step 2: Writer Agent working...")
    draft = writer_agent(research, topic)
    print(f"Draft complete! ({len(draft)} chars)")

    # Step 3: Editing
    print("\n📝 Step 3: Editor Agent working...")
    final = editor_agent(draft)
    print(f"Final version complete! ({len(final)} chars)")

    return {
        "topic": topic,
        "research": research,
        "draft": draft,
        "final": final
    }

# Run the pipeline
# result = run_content_pipeline("The benefits of meditation")
# print("\n📄 FINAL OUTPUT:")
# print(result["final"])


In [ ]:
# Define a shared state structure

from dataclasses import dataclass, field
from typing import Optional
from datetime import datetime

@dataclass
class PipelineState:
    """Shared state for the content pipeline."""
    # Input
    topic: str = ""

    # Intermediate results
    research: str = ""
    draft: str = ""
    final: str = ""

    # Metadata
    current_agent: str = ""
    history: list = field(default_factory=list)
    errors: list = field(default_factory=list)

    def log(self, agent: str, message: str):
        """Log an action to history."""
        self.history.append({
            "agent": agent,
            "message": message,
            "timestamp": datetime.now().isoformat()
        })
        self.current_agent = agent

# Test the state
state = PipelineState(topic="AI in Healthcare")
state.log("researcher", "Starting research")
print(f"Topic: {state.topic}")
print(f"History: {state.history}")

Topic: AI in Healthcare
History: [{'agent': 'researcher', 'message': 'Starting research', 'timestamp': '2026-01-30T15:15:48.368996'}]


In [ ]:
# TODO: Create agents that work with shared state

def researcher_with_state(state: PipelineState) -> PipelineState:
    """Researcher that updates shared state."""
    state.log("researcher", "Beginning research")

    try:
        # Call researcher_agent with state.topic
        state.research = researcher_agent(state.topic)
        state.log("researcher", f"Completed research ({len(state.research)} chars)")
    except Exception as e:
        state.errors.append(f"Researcher error: {e}")

    return state

def writer_with_state(state: PipelineState) -> PipelineState:
    """Writer that uses shared state."""
    state.log("writer", "Beginning writing")

    if not state.research:
        state.errors.append("Writer error: No research available")
        return state

    try:
        # Call writer_agent with state.research and state.topic
        state.draft = writer_agent(state.research, state.topic)
        state.log("writer", f"Completed draft ({len(state.draft)} chars)")
    except Exception as e:
        state.errors.append(f"Writer error: {e}")

    return state

def editor_with_state(state: PipelineState) -> PipelineState:
    """Editor that uses shared state."""
    state.log("editor", "Beginning editing")

    if not state.draft:
        state.errors.append("Editor error: No draft available")
        return state

    try:
        # Call editor_agent with state.draft
        state.final = editor_agent(state.draft)
        state.log("editor", f"Completed editing ({len(state.final)} chars)")
    except Exception as e:
        state.errors.append(f"Editor error: {e}")

    return state

print("✅ State-aware agents defined!")


✅ State-aware agents defined!


In [ ]:
# Run pipeline with shared state

def run_stateful_pipeline(topic: str) -> PipelineState:
    """Run pipeline with full state tracking."""
    # Initialize state
    state = PipelineState(topic=topic)
    state.log("orchestrator", f"Pipeline started for: {topic}")

    # Run agents in sequence
    state = researcher_with_state(state)
    state = writer_with_state(state)
    state = editor_with_state(state)

    state.log("orchestrator", "Pipeline completed")
    return state

# Run it!
# state = run_stateful_pipeline("Benefits of exercise")

# Show the history
# print("📜 PIPELINE HISTORY:")
# for entry in state.history:
#     print(f"[{entry['agent']}] {entry['message']}")

In [ ]:
# Define worker agents

def math_agent(question: str) -> str:
    """Handles math-related questions."""
    system = "You are a math expert. Solve math problems step by step. Be concise."
    return call_llm(question, system)

def science_agent(question: str) -> str:
    """Handles science-related questions."""
    system = "You are a science expert. Explain scientific concepts clearly. Be concise."
    return call_llm(question, system)

def history_agent(question: str) -> str:
    """Handles history-related questions."""
    system = "You are a history expert. Provide accurate historical information. Be concise."
    return call_llm(question, system)

def general_agent(question: str) -> str:
    """Handles general questions."""
    system = "You are a helpful assistant. Answer questions clearly and concisely."
    return call_llm(question, system)

# Map of available agents
AGENTS = {
    "math": math_agent,
    "science": science_agent,
    "history": history_agent,
    "general": general_agent
}

print("✅ Worker agents defined!")
print(f"Available agents: {list(AGENTS.keys())}")

✅ Worker agents defined!
Available agents: ['math', 'science', 'history', 'general']


In [ ]:
# TODO: Create the Supervisor Agent

def supervisor_agent(question: str) -> str:
    """Decides which agent should handle the question."""
    system = """You are a supervisor that routes questions to the right expert.

Available experts:
- math: For calculations, equations, numbers
- science: For physics, chemistry, biology, nature
- history: For historical events, dates, people
- general: For everything else

Respond with ONLY the expert name (math, science, history, or general).
No explanation, just the single word."""

    # Call the LLM to get routing decision
    response = call_llm(question, system)

    # Clean up response
    agent_name = response.strip().lower()

    # Validate - default to general if invalid
    if agent_name not in AGENTS:
        agent_name = "general"

    return agent_name


In [ ]:
# TODO: Complete Supervisor System

def supervised_qa(question: str) -> dict:
    """Full supervised Q&A system."""
    print(f"❓ Question: {question}")
    print("-" * 40)

    # Step 1: Supervisor decides
    agent_name = supervisor_agent(question)
    print(f"👔 Supervisor chose: {agent_name}")

    # Step 2: Call the chosen agent
    agent_fn = AGENTS[agent_name]
    print(f"🤖 Calling {agent_name} agent...")

    # Call the agent with the question
    answer = agent_fn(question)

    return {
        "question": question,
        "routed_to": agent_name,
        "answer": answer
    }


In [ ]:
# TODO: Build an Intent-based Router

def classify_intent(user_input: str) -> str:
    """Classify user intent into categories."""
    system = """Classify the user's intent into exactly one category:

- QUESTION: User is asking for information
- TASK: User wants something done (write, create, calculate)
- CHAT: User is making conversation
- COMPLAINT: User is expressing dissatisfaction

Respond with ONLY the category name in caps."""

    # Call LLM to classify intent
    response = call_llm(user_input, system)
    intent = response.strip().upper()

    # Validate
    valid_intents = ["QUESTION", "TASK", "CHAT", "COMPLAINT"]
    if intent not in valid_intents:
        intent = "CHAT"

    return intent

print("✅ Router defined!")


✅ Router defined!


In [ ]:
# TODO: Complete Router System

def route_and_handle(user_input: str) -> dict:
    """Route user input to appropriate handler."""
    print(f"📨 Input: {user_input}")
    print("-" * 40)

    # Classify intent
    intent = classify_intent(user_input)
    print(f"🔀 Intent: {intent}")

    # Route to handler
    handler = INTENT_HANDLERS[intent]
    response = handler(user_input)

    return {
        "input": user_input,
        "intent": intent,
        "response": response
    }


In [ ]:
# Research Team Implementation (Demo)

@dataclass
class ResearchState:
    """State for the research team."""
    query: str = ""
    research_notes: str = ""
    analysis: str = ""
    final_report: str = ""
    status: str = "initialized"
    messages: list = field(default_factory=list)

def research_team_researcher(state: ResearchState) -> ResearchState:
    """Research specialist."""
    state.messages.append("📚 Researcher: Starting research...")
    system = "You are a research specialist. Gather comprehensive information."
    state.research_notes = call_llm(f"Research: {state.query}", system)
    state.messages.append("📚 Researcher: Research complete!")
    return state

def research_team_analyst(state: ResearchState) -> ResearchState:
    """Analysis specialist."""
    state.messages.append("🔍 Analyst: Analyzing findings...")
    system = "You are an analyst. Identify key insights and patterns."
    state.analysis = call_llm(f"Analyze:\n{state.research_notes}", system)
    state.messages.append("🔍 Analyst: Analysis complete!")
    return state

def research_team_writer(state: ResearchState) -> ResearchState:
    """Report writer."""
    state.messages.append("✍️ Writer: Creating report...")
    system = "You are a report writer. Create a structured report."
    prompt = f"Topic: {state.query}\nResearch: {state.research_notes}\nAnalysis: {state.analysis}"
    state.final_report = call_llm(prompt, system)
    state.messages.append("✍️ Writer: Report complete!")
    state.status = "completed"
    return state

def run_research_team(query: str) -> ResearchState:
    """Execute the full research team workflow."""
    print(f"🔬 Research Team Activated!")
    print(f"📋 Query: {query}")
    print("=" * 50)

    state = ResearchState(query=query)

    state = research_team_researcher(state)
    print(state.messages[-1])

    state = research_team_analyst(state)
    print(state.messages[-1])

    state = research_team_writer(state)
    print(state.messages[-1])

    print(f"\n✅ Status: {state.status}")
    return state

# Run it!
# result = run_research_team("The impact of AI on employment")
# print("\n📄 FINAL REPORT:")
# print(result.final_report)